In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from glob import glob
from tqdm import tqdm

import pydensecrf.densecrf as dcrf
import pydensecrf.utils as utils

%matplotlib inline

In [ ]:
import os.path as osp

import numpy as np
import PIL.Image


def lblsave(filename, lbl):
    import imgviz

    if osp.splitext(filename)[1] != '.png':
        filename += '.png'
    # Assume label ranges [-1, 254] for int32,
    # and [0, 255] for uint8 as VOC.
    if lbl.min() >= -1 and lbl.max() < 255:
        lbl_pil = PIL.Image.fromarray(lbl.astype(np.uint8), mode='P')
        colormap = imgviz.label_colormap()
        lbl_pil.putpalette(colormap.flatten())
        lbl_pil.save(filename)
    else:
        raise ValueError(
            '[%s] Cannot save the pixel-wise class label as PNG. '
            'Please consider using the .npy format.' % filename
        )

In [ ]:
SAVE_PATH = '/scratch/ssd/gallowaa/cciw/dataset_raw/Test/Lab/done'
gmp_images = glob(osp.join(SAVE_PATH, '*gimp.png'))
print(len(gmp_images))
gmp_images

In [ ]:
# Run conditional random field post processing to retrieve missing shell pieces
MAX_ITER = 10

for i in tqdm(range(len(gmp_images))):
    img  = cv2.imread(osp.join(SAVE_PATH, gmp_images[i].split('/')[-1].split('_mask')[0] + '.jpg'))
    rgb  = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #rgb  = cv2.resize(rgb, (rgb.shape[1] // 4, rgb.shape[0] // 4))
    print(rgb.shape)
    gimp = cv2.imread(gmp_images[i])
    gimp[gimp >= 128] = 255
    gimp[gimp < 128]  = 0
    '''
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 16))
    ax1.imshow(im)
    ax2.imshow(rgb)
    '''    
    rgb = np.ascontiguousarray(rgb)
    # note the 1 - stuff seems like it should be "255 - stuff", but it seems to work better this way
    labels = np.stack([gimp[:, :, 2], 1 - gimp[:, :, 2]])
    c, h, w = labels.shape[0], labels.shape[1], labels.shape[2]
    labels = labels.astype('float') / labels.max()
    U = utils.unary_from_softmax(labels)
    U = np.ascontiguousarray(U)
    d = dcrf.DenseCRF2D(w, h, c)
    d.setUnaryEnergy(U)
    """
    @param compat=3, Potts model - it introduces a penalty for nearby similar 
    pixels that are assigned different labels. 
    """
    # This adds the color-independent term, features are the locations only.
    d.addPairwiseGaussian(sxy=3, compat=3)
    # This adds the color-dependent term, i.e. features are (x,y,r,g,b).
    # im is an image-array, e.g. im.dtype == np.uint8
    d.addPairwiseBilateral(sxy=80, srgb=13, rgbim=rgb, compat=10)
    Q = d.inference(MAX_ITER)
    Q = np.array(Q).reshape((c, h, w))
    # binarize output
    Q[0][Q[0] >= 0.5] = 1
    Q[0][Q[0] < 0.5] = 0
    crf_mask = (Q[0] * 255).astype('uint8')
    
    crf_mask_file = os.path.join(SAVE_PATH, gmp_images[i].split('/')[-1].split('.')[0] + '_final.png')
    print(crf_mask_file)
    crf_mask[crf_mask == 255] = 1
    lblsave(crf_mask_file, crf_mask) # save as indexed color RGB image